#### Names of people in the group

Thomas Bjerke

Trym Grande

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("dbfs:/FileStore/dataframes/users")
comments_df = load_df("dbfs:/FileStore/dataframes/comments")
posts_df = load_df("dbfs:/FileStore/dataframes/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df")
    sql_df = spark.sql(query)
    sql_df.show()
    print(sql_df.head()[0])
    return sql_df.head()[0]
    
def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    sql_df = spark.sql(query)
    sql_df.show()
    print(sql_df.head()[0])
    return sql_df.head()[0]

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(Id) FROM df"

q3 = "SELECT OwnerUserId from df WHERE PostTypeId = 2 GROUP BY OwnerUserId ORDER BY COUNT(OwnerUserId) DESC LIMIT 1"

q4 = "SELECT COUNT(Id) FROM df WHERE PostTypeId = 1"

q5 = "SELECT df1.DisplayName FROM df1 JOIN df2 ON df1.Id = df2.UserId GROUP BY df2.UserId, df1.DisplayName ORDER BY COUNT(df2.UserId) DESC LIMIT 1"

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

+-----+--------+----------+-------------------+-----+---------+--------------------+-----------+-------------------+-----+----+-----------+------------+-------------+---------+
|   Id|ParentId|PostTypeId|       CreationDate|Score|ViewCount|                Body|OwnerUserId|   LastActivityDate|Title|Tags|AnswerCount|CommentCount|FavoriteCount|CloseDate|
+-----+--------+----------+-------------------+-----+---------+--------------------+-----------+-------------------+-----+----+-----------+------------+-------------+---------+
|95045|   95010|         2|2021-05-29 23:28:06|    0|        0|PHA+QXMgTmlrb3Mgc...|      64377|2021-05-29 23:35:16| null|null|          0|           0|            0|     null|
+-----+--------+----------+-------------------+-----+---------+--------------------+-----------+-------------------+-----+----+-----------+------------+-------------+---------+

95045
+---------+
|count(Id)|
+---------+
|    91616|
+---------+

91616
+-----------+
|OwnerUserId|
+-----------+

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

####1.
Let's start with RDD, since they came first. A resilient distributed dataset is a collection of data that is distributed over several nodes. It is partitioned, and also replicated, which makes it faul-tolerant. Developers can perform transformations on an RDD, in which case a new copy is always made. DataFrames are also distributed collections of data, but the main difference from RDDs is that they use schemas. Every column in the dataFrame has its own name-identifier. The storage format can be e.g JSON or CSV. Another difference between RDDs and DataFrames, is that DataFrames comes with a built-in catalyst optimizer, while with RDDs, all optimization has to be done by the developers using them. Datasets can be seen as extensions of DataFrames. In addition to having name-identifiers for each column (a schema) and having a catalyst optimizer, Datasets also support type-checking very well. Also, Datasets have better object-oriented functionality. These extra features make Datasets slighly slower in aggregate operations compared with DataFrames, but it is still faster than RDDs.

####2.
RDDs should be used instead of DataFrames when working with unstructured data, that there is no point in imposing a schema on, since DataFrames require a schema. Also, RDDs are better when you want low-level actions and transformations, as DataFrames focuse more on higher-level abstractions and operations. Another benefit of RDDs is that the data can be manipulated with functional programming constructs, rather than having to use domain-specific expressions which is often the case with DataFrames. 

####3.
The main benefit of using DataSets instead of DataFrames is that with Datasets, you get a high degree of type-safety that you do not get with DataFrames.